In [1]:
import os
import re
import time
import json
import logging
import calendar
import datetime
import functools

import feedparser
from pytz import utc
from dateutil.parser import parse
import os
import re
import time
import json
import logging
import calendar
import datetime
import functools

import feedparser
from pytz import utc
from dateutil.parser import parse

try:
    from urllib2 import urlopen
    from urllib2 import HTTPError
except ImportError:
    from urllib.request import urlopen
    from urllib.error import HTTPError


# In[3]:

FEED_URL = 'http://gdacs.org/rss.aspx?profile=ARCHIVE&from={}'
DISPLACED_RE = re.compile('.*(\d+) displaced')
DEPTH_RE = re.compile('.*Depth:(\d+\.\d+)km')


def dtconv(struct_time):
    dt = datetime.datetime(*struct_time[:7], tzinfo=utc)
    return calendar.timegm(dt.utctimetuple())


def dt_to_date(s):
    dt = parse(s)
    dt = dt.replace(hour=0, minute=0, second=0)
    return calendar.timegm(dt.utctimetuple())


def tsconv(s):
    dt = parse(s)
    return calendar.timegm(dt.utctimetuple())


def get_map(entry):
    try:
        return [l['href'] for l in entry.links if l['rel'] == 'enclosure'][0]
    except IndexError:
        return None


def url_to_filename(url):
    return url.split('/')[-1]


def fetch_asset(url, outdir):
    print "fetching asset from %s" %url
    logging.debug('Fetchings asset from %s', url)
    filename = url_to_filename(url)
    outpath = os.path.join(outdir, filename)
    try:
        with open(outpath, 'w') as f:
            f.write(urlopen(url).read())
    except HTTPError:
        return ''
    return filename


def get_assets(data, outdir):
    for k, url in data.items():
        if url:
            data[k] = fetch_asset(url, outdir)


def gdacs_data(fn):
    @functools.wraps(fn)
    def wrapper(entry, outdir):
        logging.debug('Formatting entry %s', entry.gdacs_eventid)
        data = {
            'id': entry.gdacs_eventid,
            'type': entry.gdacs_eventtype,
            'name': entry.gdacs_eventname,
            'location': entry.gdacs_country or None,
            'alert_level': entry.gdacs_alertlevel.lower(),
            'updated': dtconv(entry.updated_parsed),
            'summary': entry.summary,
            'maps': {
                'thumb': get_map(entry),
                'details': entry.gdacs_mapimage or None,
            }
        }
        get_assets(data['maps'], outdir)
        data['info'] = fn(entry)
        return data
    return wrapper


@gdacs_data
def format_earthquake(entry):
    try:
        depth = float(DEPTH_RE.match(entry.summary).group(1))
    except (AttributeError, ValueError, TypeError):
        depth = None
    return {
        'severity': entry.gdacs_severity['value'],
        'affected_population': entry.gdacs_population['value'],
        'time': tsconv(entry.gdacs_fromdate),
    }


@gdacs_data
def format_flood(entry):
    return {
        'severity': entry.gdacs_severity['value'],
        'affected_population': entry.gdacs_population['value'],
        'duration': {
            'from': dt_to_date(entry.gdacs_fromdate),
            'to': dt_to_date(entry.gdacs_todate),
        },
    }


@gdacs_data
def format_tcyclone(entry):
    try:
        displaced = int(DISPLACED_RE.match(entry.summary).group(1))
    except (AttributeError, ValueError, TypeError):
        displaced = 0
    return {
        'severity': '{} {}'.format(entry.gdacs_severity['value'],
                                   entry.gdacs_severity['unit']),
        'affected_population': displaced,
        'deaths': entry.gdacs_population['value'],
        'duration': {
            'from': dt_to_date(entry.gdacs_fromdate),
            'to': dt_to_date(entry.gdacs_todate),
        },
    }


FORMATTERS = {
    'EQ': format_earthquake,
    'FL': format_flood,
    'TC': format_tcyclone,
}


def get_feed_url():
    dt = datetime.datetime.utcnow() - datetime.timedelta(days=30)
    return FEED_URL.format(dt.strftime('%Y-%m-%d'))


def get_feed():
    logging.debug('Obtaining feed')
    url = get_feed_url()
    data = feedparser.parse(url)
    return data.entries


def formatted_entries(entries, outdir):
    logging.debug('Formatting entries')
    return [FORMATTERS[e.gdacs_eventtype](e, outdir) for e in entries]


def write_json(outfile, entries):
    with open(outfile, 'w') as f:
        json.dump(entries, f, indent=2)


def main():
    import argparse
    parser = argparse.ArgumentParser(description='Convert GDACS data to JSON')
    parser.add_argument('--output', '-o', metavar='PATH', default='gdacs.json',
                        help='Output JSON file path')
    parser.add_argument('--verbose', '-V', action='store_true', help='output '
                        'debug messages')
    args = parser.parse_args()

    if args.verbose:
        logging.getLogger().setLevel(logging.DEBUG)

    entries = get_feed()
    outdir = os.path.dirname(args.output)
    write_json(args.output, formatted_entries(entries, outdir))


if __name__ == '__main__':
    main()



usage: ipykernel_launcher.py [-h] [--output PATH] [--verbose]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/michael/Library/Jupyter/runtime/kernel-bb7f1530-3ee3-4237-aee6-c340cadd5183.json


SystemExit: 2

/Users/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
who = 'http://www.who.int/feeds/entity/hac/en/rss.xml'
gdacs_eq_24h = 'http://www.gdacs.org/xml/rss_eq_24h.xml'
gdacs_all = 'http://www.gdacs.org/xml/rss.xml'
gdacs_eq_gt55_48h = 'http://www.gdacs.org/xml/rss_eq_48h_med.xml'
gdacs_all_24h = 'http://www.gdacs.org/xml/rss_24h.xml'
gdacs_all_7d = 'http://www.gdacs.org/xml/rss_7d.xml'
gdacs_ts_7d = 'http://www.gdacs.org/xml/rss_tc_7d.xml'
gdacs_ts_3m = 'http://www.gdacs.org/xml/rss_tc_3m.xml'
gdacs_fl_7d = 'http://www.gdacs.org/xml/rss_fl_7d.xml'
gdacs_fl_3m = 'http://www.gdacs.org/xml/rss_fl_3m.xml'
reliefweb = 'https://reliefweb.int/disasters/rss.xml'
reliefweb_updates = 'http://www.reliefweb.int/updates/rss.xml'
reliefweb_map = 'https://reliefweb.int/maps/rss.xml'
nasa_nh = 'http://earthobservatory.nasa.gov/Feeds/rss/nh.rss'
dost_pagasa_sl = 'https://www1.pagasa.dost.gov.ph/prsdcodes/rss/prsd/slforecast.xml'

In [7]:
outfol = '/Users/michael/Desktop/out/'
fn = fetch_asset(nasa_nh, outfol)
feed = outfol + fn
print feed

fetching asset from http://earthobservatory.nasa.gov/Feeds/rss/nh.rss
/Users/michael/Desktop/out/nh.rss


In [8]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
xml = xml.dom.minidom.parse(feed) # or xml.dom.minidom.parseString(xml_string)
xml_string = xml.toprettyxml()
print xml_string

<?xml version="1.0" ?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:georss="http://www.georss.org/georss" xmlns:media="http://search.yahoo.com/mrss/">
	
    
	<channel>
		
                                                        
		<atom:link href="https://earthobservatory.nasa.gov/feeds/natural-hazards.rss" rel="self" type="application/rss+xml"/>
		
                
		<link>https://earthobservatory.nasa.gov/feeds/natural-hazards.rss</link>
		
                                                                
		<title>NASA Earth Observatory Natural Hazards</title>
		
                                                
		<description>Your source for monitoring regional and global changes on our planet through images and stories.
        </description>
		
        
		<language>en-us</language>
		
        
		<pubDate>Tue, 18 Sep 2018 05:50:00 -0400</pubDate>
		
        
		<la

In [9]:
tree = ET.parse(feed)
root = tree.getroot()

In [10]:
for element in root[0].iter():
    print element.tag, element.attrib

channel {}
{http://www.w3.org/2005/Atom}link {'href': 'https://earthobservatory.nasa.gov/feeds/natural-hazards.rss', 'type': 'application/rss+xml', 'rel': 'self'}
link {}
title {}
description {}
language {}
pubDate {}
lastBuildDate {}
webMaster {}
image {}
title {}
url {}
link {}
width {}
height {}
description {}
item {}
title {}
link {}
{http://search.yahoo.com/mrss/}thumbnail {'url': 'https://eoimages.gsfc.nasa.gov/images/imagerecords/92000/92761/typhoonmangkhut_viirs_2018257_th.jpg'}
description {}
{http://purl.org/rss/1.0/modules/content/}encoded {}
{http://purl.org/dc/elements/1.1/}creator {}
pubDate {}
guid {}
item {}
title {}
link {}
{http://search.yahoo.com/mrss/}thumbnail {'url': 'https://eoimages.gsfc.nasa.gov/images/imagerecords/92000/92757/atlantic_geos5_2018257_th.png'}
description {}
{http://purl.org/rss/1.0/modules/content/}encoded {}
{http://purl.org/dc/elements/1.1/}creator {}
pubDate {}
guid {}
item {}
title {}
link {}
{http://search.yahoo.com/mrss/}thumbnail {'url': 

In [11]:
from xml.dom import minidom
doc = minidom.parse(feed)

for count in doc.getElementsByTagName("description"):
    print count.toxml()

<description>Your source for monitoring regional and global changes on our planet through images and stories.
        </description>
<description>Your source for monitoring regional and global changes on our planet through images and
                stories.
            </description>
<description><![CDATA[The super typhoon made landfall on the northernmost island in the Philippine archipelago.]]></description>
<description><![CDATA[The hurricane is among the longest-lived cyclones of 2018, and the storm’s winds had plenty of time to rage and ease as they ran into different environmental conditions.]]></description>
<description><![CDATA[Though flying in a lowered orbit, twelve-year-old CloudSat continues to deliver impressive science data that reveals the vertical structure of clouds.]]></description>
<description><![CDATA[Officials are planning for significant storm surges and rainfall when the storm reaches Luzon on September 15.]]></description>
<description><![CDATA[Forecasters ex

In [13]:
for elem in root.findall('.//guid'):
    print elem.tag, elem.attrib

guid {}
guid {}
guid {}
guid {}
guid {}
guid {}
guid {}
guid {}


In [11]:
from xmljson import badgerfish as bf
import overpy